In [1]:
import sys
import os
sys.path.append(os.path.abspath(r"C:\Users\valeriiz\OneDrive - ORTEC Finance\Desktop\Thesis\Code"))

import numpy as np
import pandas as pd
import time
from Methods.variable_models import Model, key_word_dict
from Methods.utils.priors import Uniform_prior, vectorize_matrix, vectorize_parameters, export_samples

In [22]:
A = np.ones([10, 30, 30])
B = np.zeros([30])+0.5


In [23]:
(A-B)

array([[[0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        ...,
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5]],

       [[0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        ...,
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5]],

       [[0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        ...,
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5]],

       ...,

       [[0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5

In [2]:
tails_external_sides=[500,500]
number_of_paths=2000
maturity_in_trend_steps=2
starting_date = None #"12/2010"
factors_from = "FSA"
prior_name= "uniform_prior"
number_of_lags=1

fixed_lower = -1
fixed_upper = 1

In [3]:
directory = "C:\\Users\\valeriiz\\OneDrive - ORTEC Finance\\Desktop\\Thesis\\Data\\priors\\"
frequencies = ["Trend", "Business cicle", "Monthly cicle"]

In [4]:
hyperparameters= pd.read_excel(directory+prior_name+"\\hyperparameters.xlsx", sheet_name=None)

In [5]:
intervals={}

for frequency in frequencies:
    upper = hyperparameters[frequency+"_UB"].to_numpy()[:, 1:]
    lower = hyperparameters[frequency+"_LB"].to_numpy()[:, 1:]

    K = upper.shape[0]

    assert K == lower.shape[0]

    upper = vectorize_matrix(upper[:,:1+K*number_of_lags]).reshape([K*(1+K*number_of_lags),1]).astype(float)
    lower = vectorize_matrix(lower[:,:1+K*number_of_lags]).reshape([K*(1+K*number_of_lags),1]).astype(float)

    upper = np.nan_to_num(upper, nan=fixed_upper)
    lower = np.nan_to_num(lower, nan=fixed_lower)

    intervals[frequency] = np.concatenate([lower, upper], axis=1)

In [6]:
priors ={}

for frequency in frequencies:
    prior = Uniform_prior(interval = intervals[frequency])

    priors[frequency] = prior

In [7]:
initial_start_time =time.time()
df=pd.read_excel(r"C:\Users\valeriiz\OneDrive - ORTEC Finance\Desktop\Thesis\Data\Thesis data set - extended data.xlsx", sheet_name="Final Data")

my_model=Model(df, start=starting_date)
number_of_dimensions=df.index.to_list()[-1]+1
print(f"num of variables {number_of_dimensions} and forecast starts at {my_model.current_date}. Import took {time.time()-initial_start_time}.")

start_time =time.time()
print("Elaborating data...", end="\r")
my_model.elaborate_data_new_1()
print(f"Elaborating data. ({time.time()-start_time})")

start_time =time.time()
print("Decomposing into 3 frequencies...", end="\r")
my_model.decompose(tails_external_sides=tails_external_sides)
print(f"Decomposing into 3 frequencies. ({time.time()-start_time})")

start_time =time.time()
print("Elaborating data...", end="\r")
my_model.elaborate_data_2()
print(f"Elaborating data. ({time.time()-start_time})")

start_time =time.time()
print("Standardizing components...", end="\r")
my_model.standardize_decomposed_dict("Decomposed df LogReturns_F")
print(f"Standardizing components. ({time.time()-start_time})")

start_time =time.time()
factors = {}

for f_and_t in my_model.frequency_names:
    
    names=pd.read_excel(f"C:\\Users\\valeriiz\\OneDrive - ORTEC Finance\\Desktop\\Thesis\\Data\\Factors\\O_FSA_Factors_{f_and_t}.xlsx", sheet_name="Factors")["Factor"].to_list()

    factors[f_and_t]=my_model.decomposed_dict["DR_input"][f_and_t].copy().rename(columns={"Asset": "Factor"}).drop(columns=["Datashape"])

    factors[f_and_t]=factors[f_and_t][factors[f_and_t]["Factor"].isin(names)]

    factors[f_and_t] = factors[f_and_t].dropna(how="any", axis=0)

    factors[f_and_t]['Order'] = factors[f_and_t]['Factor'].map({name: i for i, name in enumerate(names)})

    factors[f_and_t] = factors[f_and_t].sort_values('Order').drop('Order', axis=1)
    
    factors[f_and_t].reset_index(drop=True, inplace=True)

print("Reducing dimensionality...", end="\r")
my_model.do_FSA(factors_pool=factors, max_expl=10, fixed_order=True)
my_model.do_PCA()
print(f"Reducing dimensionality. ({time.time()-start_time})")

components = {"Trend": 3,"Business cicle": 9,"Monthly cicle": 10 }
jumps = { "Trend": 96,"Business cicle": 12,"Monthly cicle": 1 }
lags = { "Trend": 1,"Business cicle": 1,"Monthly cicle": 1 }

num of variables 300 and forecast starts at 6/2024. Import took 4.5936808586120605.
Elaborating data. (0.14461588859558105)
Decomposing into 3 frequencies. (19.00502920150757)
Elaborating data. (1.7409305572509766)
Standardizing components. (0.14121031761169434)


C:\Users\valeriiz\OneDrive - ORTEC Finance\Desktop\Thesis\Code\Methods\utils\utils_algorithm.py:179: UserWarning: Capping phase at 3.
  warnings.warn(f"Capping phase at {M}.")
C:\Users\valeriiz\OneDrive - ORTEC Finance\Desktop\Thesis\Code\Methods\utils\utils_algorithm.py:179: UserWarning: Capping phase at 9.
  warnings.warn(f"Capping phase at {M}.")


Reducing dimensionality. (0.5093533992767334)


In [8]:
folder_name = f"{factors_from}_lag_{number_of_lags}_w_{number_of_paths}_to_{starting_date}"

for frequency in frequencies:

    data = my_model.decomposed_dict[factors_from][frequency][key_word_dict[factors_from]][:components[frequency]]

    priors[frequency].aggregate_likelihood(data, number_of_lags= number_of_lags, jumps= jumps[frequency])

    sample_dict = priors[frequency].sample(2000)

    loc = directory + prior_name+"\\samples\\" + frequency + f"\\{folder_name}"

    if not os.path.exists(loc):
        os.mkdir(loc)
        print(f"Directory '{loc}' created.")
    else:
        print(f"Directory '{loc}' already exists.")

    export_samples(loc, sample_dict)

Directory 'C:\Users\valeriiz\OneDrive - ORTEC Finance\Desktop\Thesis\Data\priors\uniform_prior\samples\Trend\FSA_lag_1_w_2000_to_None' already exists.
Directory 'C:\Users\valeriiz\OneDrive - ORTEC Finance\Desktop\Thesis\Data\priors\uniform_prior\samples\Business cicle\FSA_lag_1_w_2000_to_None' created.
Directory 'C:\Users\valeriiz\OneDrive - ORTEC Finance\Desktop\Thesis\Data\priors\uniform_prior\samples\Monthly cicle\FSA_lag_1_w_2000_to_None' created.
